# Locate point cloud tiles in storage based on target areas

In [3]:
import pandas as pd
import geopandas as gpd
from tqdm.notebook import tqdm
tqdm.pandas()
import shapely.geometry as sg

In [18]:
### SETTINGS ###
run = 'run1'  # or 'run2'
data_file = f'../datasets/AMS_{run}_tiles_list.csv'


In [19]:
all_tiles = pd.read_csv(data_file)
all_tiles.head()

,folder,tilecode,n_points,RD_X,RD_Y,dup
0,nl-amsd-200519-7415-laz,2201_9810,1815,110050,490500,False
1,nl-amsd-200519-7415-laz,2340_9815,61943,117000,490750,False
2,nl-amsd-200519-7415-laz,2340_9814,40536,117000,490700,False
3,nl-amsd-200519-7415-laz,2340_9813,588,117000,490650,False
4,nl-amsd-200519-7415-laz,2340_9809,1073180,117000,490450,False


In [20]:
# Convert to GeoDataFrame
all_tiles['geometry'] = all_tiles.apply(lambda row: sg.box(row.RD_X, row.RD_Y, row.RD_X+50, row.RD_Y+50), axis=1)
all_tiles_gdf = gpd.GeoDataFrame(all_tiles)
all_tiles = None

In [43]:
def get_tile_code(coord):
    target_points = gpd.GeoDataFrame({'geometry': [sg.Point(coord)]})
    all_tiles_gdf['target'] = all_tiles_gdf.apply(lambda row: target_points.intersects(row.geometry).any(), axis=1)
    target_tiles_gdf = all_tiles_gdf[all_tiles_gdf.target]
    return target_tiles_gdf['tilecode'].values[0]

In [45]:
df_lamps = pd.read_csv('armatuur_table.csv')
df_lamps['tile_code'] = df_lamps.apply(lambda row: get_tile_code([row.coord_X,row.coord_Y]), axis=1)
df_lamps.head()

In [ ]:
df_lamps.to_csv('armatuur_table.csv', index=False)